## 步骤1：创建数据
Tensors张量是一种特殊的数据结构，它和数组还有矩阵十分相似。在Pytorch中，Tensors可以在gpu或其他专用硬件上运行来加速计算之外，其他用法类似Numpy。

In [ ]:
import torch
import numpy as np
# 直接从数据创建
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)
x_data.shape
# 全为1
x_ones = torch.ones_like(x_data)  # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

# 全为0
x_rand = torch.rand_like(x_data, dtype=torch.float)  # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

# 查看tensor类型
tensor = torch.rand(3, 4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

## 步骤2：自动梯度计算

在Pytorch中可以使用tensor进行计算，并最终可以从计算得到的tensor计算损失，并进行梯度信息。在Pytorch中主要关注正向传播的计算即可。

In [ ]:
# x = torch.ones(2, 2, requires_grad=True)
x = torch.tensor([[1, 2], [3, 4]], dtype=float, requires_grad=True)
print(x)

y = x + 2
print(y)
print(y.grad_fn)  # y就多了一个AddBackward

z = y * y * 3
out = z.mean()

print(z)  # z多了MulBackward
print(out)  # out多了MeanBackward

# 计算公式：out = 0.25 ((x+2) * (x+2) * 3)
out.backward()
print(x.grad)

## 步骤3：拟合曲线

接下来我们将尝试使用Pytorch拟合一条曲线，我们首先的创建待你和的参数，并加载待训练的数据。

In [ ]:
# 需要计算得到的参数
w = torch.ones(1, requires_grad=True)
b = torch.ones(1, requires_grad=True)

# 数据
x_tensor = torch.from_numpy(x)
y_tensor = torch.from_numpy(y)

# 目标模型
# y = wx + b
# 定义损失
def mse(label, pred):
    diff = label - pred
    return torch.sqrt((diff ** 2).mean())

pred = x_tensor * w + b
loss = mse(y_tensor, pred)
# 执行20次参数更新
for _ in range(20):

    # 重新定义一下，梯度清空
    w = w.clone().detach().requires_grad_(True)
    b = b.clone().detach().requires_grad_(True)

    # 正向传播
    pred = x_tensor * w + b
    
    # 计算损失
    loss = mse(y_tensor, pred)
    print(loss)

    # 计算梯度
    loss.backward()

## 步骤4：加载MNIST数据集

torchvision是pytorch官方的用于视觉任务的库，这里我们加载最常见的MNST数据集。当然也可以自定义数据读取

In [ ]:
# torchvision 是pytorch官方的用于视觉任务的库
import torchvision.datasets as datasets  # 内置的数据集读取
import torchvision.transforms as transforms  # 内置的对图像的操作
from torch import nn

# 组合多个数据变换
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 图片的读取，（图片、类别）
# 28 * 28，数字0、1、2、3、4、5、6、7、8、9
dataset1 = datasets.MNIST('./', train=True, download=True)
dataset2 = datasets.MNIST('./', train=False, download=True)

train_loader = torch.utils.data.DataLoader(dataset1, batch_size=40)
test_loader = torch.utils.data.DataLoader(dataset2,  batch_size=40)

## 步骤5：定义全连接网络

接下来我们定义网络结构，由于是图像分类任务，因此我们的节点维度使用逐步降低的定义。

In [ ]:
net = nn.Sequential(
    nn.Flatten(), # 将维度转换为二维
    nn.Linear(784, 256), # 全连接层
    nn.ReLU(), # 激活函数
    nn.Linear(256, 10) # 全连接层
)

## 步骤6：训练卷积神经网络

如果需要定义CNN网络，则可以参考如下的方式。先定义卷积层，然后定义全连接层。

In [ ]:
import torch
from torch import nn

class Reshape(torch.nn.Module):
    def forward(self, x):
        return x.view(-1, 1, 28, 28)

net = torch.nn.Sequential(
    Reshape(),
    nn.Conv2d(1, 6, kernel_size=5, padding=2),
    nn.ReLU(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5),
    nn.ReLU(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120),
    nn.ReLU(),
    nn.Linear(120, 84),
    nn.ReLU(),
    nn.Linear(84, 10)
)

## 步骤7：模型训练

定义训练时的超参数，如batch size、学习率和优化器。这里可以自定定义。

In [ ]:
batch_size, lr, num_epochs = 256, 0.1, 10
loss = nn.CrossEntropyLoss()
updater = torch.optim.SGD(net.parameters(), lr=lr)
train_acc, test_acc = [], []
# epoch维度训练
for _ in range(num_epochs):
    acc = 0
    
    # 读取训练数据
    # batch维度训练
    for data in train_loader:
        pred = net(data[0]) # 正向传播
        pred_loss = loss(pred, data[1]) # 计算损失
        updater.zero_grad() # 清空梯度
        pred_loss.backward()  # 梯度计算
        updater.step()  # 参数更新
        
        # 累计准确样本个数
        acc += (pred.argmax(1) == data[1]).sum()
    
    # 计算准确率
    acc = acc.float() / len(train_loader.dataset)
    train_acc.append(acc)
    
    # 读取验证数据
    # batch维度预测
    with torch.no_grad(): # 不记录梯度信息
        acc = 0
        for data in test_loader:
            pred = net(data[0]) # 正向传播
            pred_loss = loss(pred, data[1]) # 累计梯度
            acc += (pred.argmax(1) == data[1]).sum() # 累计准确样本个数
        
        # 计算准确率
        acc = acc.float() / len(test_loader.dataset)
        test_acc.append(acc)

    print(train_acc[-1], test_acc[-1])